__CrossEntropyLoss详解__

In [35]:
import torch
import torch.nn.functional as F

#生成神经网络output，一行对应一个样本，一列对应该样本属于该列类的预测概率
output = torch.randn(5, 5)
print('output:\n',output)

#生成5个样本的标签，并转换为One-Hot编码
target = torch.tensor([0, 2, 1, 3, 4])
one_hot = F.one_hot(target).float()
print('one_hot:\n',one_hot)

#softmax层运算，每个通道每个像素点的值转换为预测概率值
softmax = torch.exp(output)/torch.sum(torch.exp(output),dim=1).reshape(-1, 1)
print('softmax:\n',softmax)

#对数运算，保证数值稳定性和overflaw、underflaw
logsoftmax = torch.log(softmax)
print('logsoftmax:\n',logsoftmax)

#按照label标号为1的像素点位置读取logsoftmax，计算损失函数值
nllloss = -torch.sum(one_hot*logsoftmax)/target.shape[0]
print('one_hot*logsoftmax:\n',one_hot*logsoftmax)
print('nllloss:\n',nllloss)

#直接使用封装好的CrossEntropyLoss计算损失函数值
loss = torch.nn.CrossEntropyLoss()
value = loss(output, target)
print('loss:\n',value)

output:
 tensor([[ 0.6522,  0.8080,  0.2059,  0.3446, -0.2827],
        [-1.1521, -0.6535, -0.3348, -1.6790,  1.5831],
        [ 1.2938, -1.3619,  0.3224,  0.2244, -0.2479],
        [-0.3655,  0.6795,  1.2152,  1.8519, -0.2604],
        [ 0.9388, -0.2316, -0.8106, -1.9803, -1.1706]])
one_hot:
 tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])
softmax:
 tensor([[0.2541, 0.2969, 0.1626, 0.1868, 0.0997],
        [0.0478, 0.0787, 0.1083, 0.0282, 0.7370],
        [0.4985, 0.0350, 0.1887, 0.1711, 0.1067],
        [0.0526, 0.1497, 0.2558, 0.4834, 0.0585],
        [0.6026, 0.1870, 0.1048, 0.0325, 0.0731]])
logsoftmax:
 tensor([[-1.3702, -1.2145, -1.8166, -1.6779, -2.3052],
        [-3.0404, -2.5418, -2.2232, -3.5673, -0.3052],
        [-0.6962, -3.3519, -1.6675, -1.7656, -2.2379],
        [-2.9442, -1.8992, -1.3635, -0.7268, -2.8392],
        [-0.5065, -1.6769, -2.2558, -3.4255, -2.6158]])
one_

__咽喉反流CB_Loss__

In [ ]:
import torch
import math
def cb_loss(y_pred, y_true, temp, beta, confusion):
    confusion_ = [torch.sum(temp == i) for i in range(8)]   #统计一张label中每个类别的像素点

    weight = []
    weight_dice = []
    for i, n in zip(confusion_, confusion):
        if i == 0:
            weight.append(0)
            weight_dice.append(1)
        else:
            weight_dice.append(1)
            weight.append(((1.0 - beta) / (1.0 - math.pow(beta, n))))
    
    
    weight = torch.FloatTensor(weight)
    weight_dice = torch.FloatTensor(weight_dice)
    weight = weight.to(y_pred.device)
    weight_dice = weight_dice.to(y_pred.device)


    criterion_train = torch.nn.CrossEntropyLoss(weight=weight)

    loss = criterion_train(y_pred.float(), temp.float())

    return loss

__CB_Loss__

In [ ]:
import math
import numpy as np

def cb_loss(y_pred, y_true, beta, confusion):
    weight = [((1.0 - beta) / (1.0 - math.pow(beta, i))) for i in confusion]
    criterion_train = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(weight))
    loss = criterion_train(y_pred.float(), y_true.long())
    return loss

def train_for_cb_loss(model, optimizer)
    cls = []
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ") #delimiter-定界符

